# Limpieza de Datos

En este notebook, realizaremos la limpieza y preprocesamiento de los datos extraídos de la API de Finca Raíz. Esto incluye la eliminación de valores nulos, la conversión de tipos de datos y la creación de nuevas características que serán útiles para el modelado.

In [ ]:
# Importar librerías necesarias
# -*- coding: utf-8 -*-
import pandas as pd

from sklearn.impute import KNNImputer
import re
from datetime import datetime
from ast import literal_eval
from sklearn.preprocessing import MultiLabelBinarizer
import unicodedata
from textblob import TextBlob
import unidecode 
import numpy as np
from datetime import datetime


# Cargar datos crudos
raw_data_path = '../data/raw/propiedades_fincaraiz_completas.csv'   #'..data/processed/data_arriendos_model.xlsx'  
df_arriendos = pd.read_csv(raw_data_path, sep=';', encoding='utf-8-sig')

# Mostrar las primeras filas del conjunto de datos
df_arriendos.head()

In [ ]:
# Eliminar donde departamento es diferente de antioquia
df_arriendos = df_arriendos[df_arriendos['departamento'] == 'Antioquia']

#Filtra los datos que tengan un precio mayor a 50000000
df_arriendos = df_arriendos[df_arriendos['precio'] <= 50000000]


## Si estrato es 100 es campestre se puede reemplazar por 3, 
# si es 110 son 'sin definir' se puede reemplazar por na, 
# los de 0 se pueden reemplazar por np.nan

df_arriendos['estrato'] = df_arriendos['estrato'].replace(100, 3)
df_arriendos['estrato'] = df_arriendos['estrato'].replace(110, np.nan)
df_arriendos['estrato'] = df_arriendos['estrato'].replace(0, np.nan)

# Remplazar antiguedad null por "no disponible"
df_arriendos['antigüedad'] = df_arriendos['antigüedad'].fillna('No disponible')

In [ ]:
# Eliminar duplicados, nulos en precio, columnas no utiles, limpia el dataset, organizar tipos

df_arriendos = df_arriendos.drop(columns=['redes_sociales', 'video', 'telefono', 'direccion_propietario', 'tipo_propietario', 'propietario',
                                          'url_anuncio', 'imagenes','moneda', 'direccion', 'id', 'titulo', 'departamento', 'barrio'])
df_arriendos = df_arriendos.dropna(subset=['precio'])

In [ ]:
# Función para limpiar y convertir a numérico
def limpiar_area(valor):
    if isinstance(valor, str):  # Verificar si es un string
        valor = valor.replace(" m2", "").strip()  # Eliminar el texto "m2" y espacios adicionales
        return pd.to_numeric(valor, errors='coerce')  # Convertir a float, dejando NaN si falla
    return valor  # Devolver sin cambios si ya es numérico o NaN

# Aplicar la limpieza a las columnas de área
df_arriendos['area_construida_m2'] = df_arriendos['area_construida_m2'].apply(limpiar_area)
df_arriendos['area_privada_m2'] = df_arriendos['area_privada_m2'].apply(limpiar_area)

In [ ]:
# Filtrar na de area_construida_m2
df_arriendos = df_arriendos.dropna(subset=['area_construida_m2'])

# Remplazar los valores de area_privada_m2 que sean na por el valor de area_construida_m2
df_arriendos.loc[df_arriendos['area_privada_m2'].isna(), 'area_privada_m2'] = df_arriendos['area_construida_m2']

# Para habitaciones de mas de 5 el area debe ser mayor a habitaciones*10
df_arriendos = df_arriendos[~((df_arriendos['habitaciones'] > 5) & (df_arriendos['area_construida_m2'] < df_arriendos['habitaciones']*10))]


In [ ]:
#Eliminar precios  menores o iguales a 500 mil
df_arriendos = df_arriendos[df_arriendos['precio'] >= 500000]

#Reemplazar baños y habitaciones en cero por np.nan
df_arriendos['baños'] = df_arriendos['baños'].replace(0, np.nan)
df_arriendos['habitaciones'] = df_arriendos['habitaciones'].replace(0, np.nan)

# Eliminar área construida menor a 9 m2
df_arriendos = df_arriendos[df_arriendos['area_construida_m2'] >= 9]

#Eliminar área privada menor a 30 m2
df_arriendos = df_arriendos[df_arriendos['area_privada_m2'] >= 9]

#Si el area privada es mayor al area construida se reemplaza el area construida por el area privada
df_arriendos['area_construida_m2'] = np.where(df_arriendos['area_construida_m2'] < df_arriendos['area_privada_m2'], df_arriendos['area_privada_m2'], df_arriendos['area_construida_m2'])

#Si el area privada es menos de la mitad del area construida se reemplaza el area privada por el area construida
df_arriendos['area_privada_m2'] = np.where(df_arriendos['area_privada_m2'] < df_arriendos['area_construida_m2']/2, df_arriendos['area_construida_m2'], df_arriendos['area_privada_m2'])

In [ ]:
#Si pisos_edificio es 0 y piso_ubicacion >0 entonces colocar piso ubicación 
df_arriendos['pisos_edificio'] = np.where((df_arriendos['pisos_edificio'] == 0) & (df_arriendos['piso_ubicacion'] > 0), df_arriendos['piso_ubicacion'], df_arriendos['pisos_edificio'])

# Si ambos son 0 colocar 1 en ambos
df_arriendos['pisos_edificio'] = np.where((df_arriendos['pisos_edificio'] == 0) & (df_arriendos['piso_ubicacion'] == 0), 1, df_arriendos['pisos_edificio'])
df_arriendos['piso_ubicacion'] = np.where((df_arriendos['pisos_edificio'] == 0) & (df_arriendos['piso_ubicacion'] == 0), 1, df_arriendos['piso_ubicacion'])

#Si el piso de ubicación es mayor al número de pisos del edificio se reemplaza por el número de pisos del edificio
df_arriendos['piso_ubicacion'] = np.where(df_arriendos['piso_ubicacion'] > df_arriendos['pisos_edificio'], df_arriendos['pisos_edificio'], df_arriendos['piso_ubicacion'])

# Reemplazar en piso si es mayor a 100 tomar el primer numero (401 y 503 por 4, 5)
df_arriendos['piso_ubicacion'] = df_arriendos['piso_ubicacion'].apply(lambda x: int(str(x)[0]) if x > 100 else x)
df_arriendos['pisos_edificio'] = df_arriendos['pisos_edificio'].apply(lambda x: int(str(x)[0]) if x > 100 else x)


In [ ]:
# Si el precio es menor a 5 millones y parqueaderos >= 5  y es apartamento entonces imputar por 2
df_arriendos['parqueaderos'] = np.where((df_arriendos['precio'] < 5000000) & (df_arriendos['parqueaderos'] >= 5) & (df_arriendos['tipo_de_inmueble'] == 'Apartamento'), 2, df_arriendos['parqueaderos'])

#Si precio mayor a 5 millones y parqueaderos >= 5 entonces 6 
df_arriendos['parqueaderos'] = np.where((df_arriendos['precio'] >= 5000000) & (df_arriendos['parqueaderos'] > 5), 6, df_arriendos['parqueaderos'])

In [ ]:
#Imputar con Modelos Predictivos (KNN Imputer)

imputer = KNNImputer(n_neighbors=5)
df_arriendos[['habitaciones', 'baños','estrato']] = imputer.fit_transform(df_arriendos[['habitaciones', 'baños','estrato']])
df_arriendos[['habitaciones', 'baños','estrato']] = df_arriendos[['habitaciones', 'baños','estrato']].apply(np.floor).astype(int)

In [ ]:
# Reemplazar estado vacias por estandar 
df_arriendos['estado'] = df_arriendos['estado'].fillna('Estandar')

In [ ]:
# Para la fecha calcular los dias, meses desde publicado y actaulizado

df_arriendos['fecha_actualizacion'] = pd.to_datetime(df_arriendos['fecha_actualizacion'])
df_arriendos['dias_desde_actualizado'] = (datetime.now() - df_arriendos['fecha_actualizacion']).dt.days

df_arriendos = df_arriendos.drop(columns=['fecha_publicacion', 'fecha_actualizacion'])

In [ ]:
## Si no tiene administracion entonces 0
df_arriendos['gastos_comunes'] = df_arriendos['gastos_comunes'].fillna(0)

In [ ]:
# Filtramos si la ciudad esta en ciudades definidas
ciudades = ["Medellín", "Sabaneta", "Envigado", "Itagüí", "Bello", "La estrella", "Caldas", "Copacabana", "Girardota", "Barbosa"]

df_arriendos = df_arriendos[df_arriendos['ciudad'].isin(ciudades)]

In [ ]:
# reemplazar facilidades vacias por lista ['No disponible']
df_arriendos['facilidades'] = df_arriendos['facilidades'].fillna('No disponible')

In [ ]:
def agrupar_caracteristicas(df, columna):
    """
    Reduce la dimensionalidad de características del DataFrame agrupando términos similares en categorías más generales.
    Genera nuevas columnas binarias para cada categoría (1 si al menos un elemento de la categoría está presente, 0 si no).
    """
    categorias_agrupadas = {
        "Infraestructura y Accesibilidad": ["Acceso Pavimentado", "Cómodas vías de acceso", "Sobre vía principal", "Sobre vía secundaria", "Trans. Público cercano"],
        "Parqueadero": ["Garaje", "Garaje Cubierto", "Parqueadero(s)", "Parqueadero Subterráneo", "Parqueadero Visitantes", "Parqueadero inteligente", "Bahía exterior de parqueo"],
        "Seguridad": ["Alarma", "Alarma Contra Incendio", "Circuito cerrado de TV", "Control de acceso digital", "Reja de Seguridad", "Puerta de seguridad", "Vigilancia 24x7", "Seguridad", "Estación de Policía", "Guardia"],
        "Ascensores": ["Ascensor", "Ascensores Comunales", "Ascensor(es) inteligente(s)", "Ascensor Privado"],
        "Servicios Públicos": ["Alcantarillado", "Tanques de Agua", "Planta Eléctrica", "Shut de basura"],
        "Tecnología y Conectividad": ["Disponibilidad WiFi", "Cableado de Red", "Detector de Metales", "Sensor de movimiento", "Edificio Inteligente"],
        "Distribución": ["Balcones", "Terraza", "Patio", "Patio Interno"],
        "Baños": ["Baño Auxiliar", "Baño de Servicio", "Baño Independiente", "Baño compartido"],
        "Cocina": ["Cocina Equipada", "Cocina Integral", "Cocina tipo Americano", "Cocineta", "Barra estilo americano"],
        "Zonas Sociales": ["Salón Comunal", "Salón de Juegos", "Salón de conferencias", "Salón de videoconferencias", "Gimnasio"],
        "Otras Características": ["Chimenea", "Estudio", "Mezzanine", "Loft", "Duplex"],
        "Zona Residencial": ["Zona Residencial", "En zona residencial", "Ubicada en edificio", "En conjunto cerrado", "En condominio"],
        "Zona Comercial": ["Zona Comercial", "Cerca a sector comercial", "Pasaje Comercial", "En centro Comercial", "Locales comerciales", "Oficinas administrativas"],
        "Zona Industrial": ["Zona Industrial", "Parque industrial", "Bodega", "Galpón", "Soporte de grúas"],
        "Zona Campestre": ["Zona Campestre", "Área Rural", "Finca cafetera", "Nacimientos de agua", "Río / Quebrada cercano(a)"],
        "Deportes": ["Cancha de Baloncesto", "Cancha de Futbol", "Cancha de Squash", "Cancha de Tennis", "Canchas Deportivas"],
        "Piscina y Spa": ["Piscina", "Sauna / Turco / Jacuzzi"],
        "Zonas Verdes": ["Jardines Exteriores", "Zonas Verdes", "Senderos ecológicos", "Árboles frutales"],
        "BBQ y Entretenimiento": ["Zona de BBQ", "Kiosko", "Zona de Camping", "Zona de Hamacas"],
        "Condiciones del Inmueble": ["Remodelado", "Para estrenar", "Amoblado", "Moderno"],
        "Exclusividad y Lujo": ["Exclusivo", "Lujoso", "Premium", "Alta Gama", "Penthouse"],
        "Vista Panorámica": ["Vista panorámica", "Hermosa vista", "Vista a la ciudad", "Vista a las montañas"],
        "Confortabilidad": ["Espacioso", "Amplio", "Luminoso", "Acogedor"],
        "Transporte": ["Proximo al metro", "Transporte público", "vías principales","cerca del metro", "cercano al metro", "metroplus", "ruta integrada", "transporte publico"]
    }

    # Convertir todas las categorías a minúsculas y sin acentos para mejorar la comparación
    categorias_agrupadas = {
        categoria: {unidecode.unidecode(item.lower()) for item in items}
        for categoria, items in categorias_agrupadas.items()
    }

    # Asegurar que la columna de entrada sea una lista de valores en cada fila
    df_arriendos['facilidades'] = df_arriendos['facilidades'].apply(lambda x: x if isinstance(x, list) else literal_eval(x))
    df_arriendos['facilidades'] = df_arriendos['facilidades'].apply(lambda x: literal_eval(x) if isinstance(x, str) else x)
    df_arriendos['facilidades'] = df_arriendos['facilidades'].apply(lambda x: x if isinstance(x, list) else [])
    df_arriendos['facilidades'] = df_arriendos['facilidades'].apply(lambda x: [item for item in x if item is not None])

  
    # Crear una columna binaria por cada categoría
    for categoria, elementos in categorias_agrupadas.items():
        df[categoria] = df[columna].apply(lambda facilidades: 1 if any(unidecode.unidecode(item.lower()) in elementos for item in facilidades) else 0)
    return df


df_arriendos = agrupar_caracteristicas(df_arriendos, 'facilidades')


In [ ]:
##Normalizar texto de la descripción

def normalizar_texto(texto):
    texto = texto.lower()  # Convertir a minúsculas
    texto = unicodedata.normalize("NFKD", texto).encode("ascii", "ignore").decode("utf-8")  # Eliminar tildes
    texto = re.sub(r'[^\w\s]', '', texto)  # Eliminar signos de puntuación
    return texto

# Aplicamos normalización a la descripción
df_arriendos["descripcion_normalizada"] = df_arriendos["descripcion"].apply(normalizar_texto)

In [ ]:

# Diccionario de sinónimos y palabras clave
sinonimos = {
    "Transporte 2": ["cerca del metro", "cercano al metro", "proximo al metro", "a pocos pasos del metro", "transporte publico", "via principal", "buses"],
    "Exclusividad 2": ["exclusivo", "lujoso", "alta gama", "premium", "espectacular", "de lujo", 'Penthouse'],
    "Seguridad 2": ["seguridad", "circuito cerrado", "estacion de policia", "guardia", "vigilancia"],
    "Vista Panorámica 2": ["vista panoramica", "vista espectacular", "hermosa vista", "gran vista"],
    "Confortabilidad 2": ["espacioso", "amplio", "hermoso", "luminoso", "acogedor"]
}

# Lista de condiciones de la vivienda
condiciones = ['remodelado', 'para estrenar', 'amoblado', 'moderno']

# Función para detectar características
def detectar_caracteristica(texto, palabras_clave):
    texto = texto.lower()
    return int(any(palabra in texto for palabra in palabras_clave))

# Aplicar la detección de características

for categoria, palabras in sinonimos.items():
    df_arriendos[categoria] = df_arriendos['descripcion_normalizada'].apply(lambda x: detectar_caracteristica(x, palabras))

for condicion in condiciones:
    df_arriendos[condicion] = df_arriendos['descripcion_normalizada'].str.contains(condicion, case=False, na=False).astype(int)

# Análisis de sentimiento
df_arriendos['sentimiento'] = df_arriendos['descripcion_normalizada'].apply(lambda x: TextBlob(x).sentiment.polarity)


df_arriendos.drop(columns=['descripcion', 'descripcion_normalizada'], inplace=True)

In [ ]:
# Agrupación de Facilidades en Categorías

# Ubicación y accesibilidad: Transporte, cerca de centros comerciales, zona comercial.
# Seguridad: Vigilancia, circuito cerrado de TV, rejas de seguridad.
# Amenidades y confort: Gimnasio, piscina, zona BBQ.
# Condición del apartamento: Remodelado, para estrenar, amoblado.

df_arriendos['Seguridad'] = df_arriendos[['Seguridad', 'Seguridad 2']].max(axis=1)
df_arriendos['Transporte'] = df_arriendos[['Transporte', 'Transporte 2']].max(axis=1)
df_arriendos['Vista Panorámica'] = df_arriendos[['Vista Panorámica', 'Vista Panorámica 2']].max(axis=1)
df_arriendos['Exclusividad'] = df_arriendos[['Exclusividad y Lujo', 'Exclusividad 2']].max(axis=1)
df_arriendos['Confortabilidad'] = df_arriendos[['Confortabilidad', 'Confortabilidad 2']].max(axis=1)
df_arriendos['amenidades'] = df_arriendos[['remodelado', 'para estrenar', 'amoblado', 'moderno']].max(axis=1)

# Eliminar las columnas originales
df_arriendos.drop(columns=['Seguridad 2', 'Transporte 2', 'Vista Panorámica 2', 'Exclusividad 2', 'Confortabilidad 2',
                           'remodelado', 'para estrenar', 'amoblado', 'moderno'], inplace=True)

In [ ]:
categorias_agrupadas = [
"Infraestructura y Accesibilidad", 
"Parqueadero", 
"Seguridad", 
"Ascensores",
"Servicios Públicos",
"Tecnología y Conectividad",
"Distribución", 
"Baños", 
"Cocina",  
"Zonas Sociales", 
"Otras Características", 
"Zona Residencial", 
"Zona Comercial", 
"Zona Industrial", 
"Zona Campestre", 
"Deportes",
"Piscina y Spa", 
"Zonas Verdes", 
"BBQ y Entretenimiento", 
"Condiciones del Inmueble", 
"Exclusividad y Lujo", 
"Vista Panorámica", 
"Confortabilidad", 
"Transporte", 
]
facilidades_encoded = df_arriendos[categorias_agrupadas]
#quitar las columnas de facilidades
df_arriendos  = df_arriendos.drop(columns=categorias_agrupadas)

In [ ]:
# Análisis de Frecuencia de Facilidades

# Filtrar facilidades con baja frecuencia (aparecen en menos del 1% de los registros)
frecuencia = facilidades_encoded.mean()
facilidades_filtradas = frecuencia[frecuencia > 0.01].index.tolist()
df_filtrado = facilidades_encoded[facilidades_filtradas]

# Unir al dataframe original
df_arriendos = df_arriendos.drop(columns=['facilidades']).join(df_filtrado)

In [ ]:
#Reestructurar columnas de df_arriendos
df_arriendos.columns = (
    df_arriendos.columns
    .str.strip()  # Eliminar espacios en los extremos
    .str.lower()  # Convertir a minúsculas
    .to_list()  # Convertir el Index a una lista
)

#Ahora podemos aplicar unidecode a cada nombre de columna
df_arriendos.columns = [unidecode.unidecode(col) for col in df_arriendos.columns]

#Reemplazar caracteres no alfanuméricos
df_arriendos.columns = df_arriendos.columns.str.replace(r'\W+', '_', regex=True)

In [ ]:
#Eliminar nulos
df_arriendos = df_arriendos.dropna()

#Eliminar duplicados
df_arriendos = df_arriendos.drop_duplicates()

# Guardar el archivo final
df_arriendos.to_excel('../data/processed/data_arriendos_model.xlsx', index=False)

## Conclusión

Hemos limpiado los datos eliminando filas con valores nulos y convirtiendo tipos de datos. También hemos creado una nueva característica que puede ser útil para el modelado. Los datos procesados se han guardado para su uso en análisis posteriores.